In [1]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [2]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-f98f09b7-9082-4200-ba38-2baeba1186cb'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [3]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [4]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match('EUROPE', matchId)

    match = {}
    match['matchId'] = matchId
    match['gameDuration'] = match_detail['info']['gameDuration']
    match['gameType'] = match_detail['info']['gameType']
    match['gameVersion'] = match_detail['info']['gameVersion']
    matches.append(match)

    for index, row in enumerate(match_detail['info']['participants']):
        participants_row = {}
        participants_row['matchId'] = matchId
        participants_row['participant'] = index + 1
        participants_row['summonerName'] = row['summonerName']
        participants_row['teamPosition'] = row['teamPosition']
        participants_row['championName'] = row['championName']
        participants_row['kda'] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
        participants_row['champLevel'] = row['champLevel']
        participants_row['totalDamageToChampions'] = row['totalDamageDealtToChampions']
        participants_row['totalDamageTaken'] = row['totalDamageTaken']
        participants_row['totalHeal'] = row['totalHeal']
        participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
        participants_row['goldEarned'] = row['goldEarned']
        participants_row['summoner1'] = row['summoner1Id']
        participants_row['summoner2'] = row['summoner2Id']
        participants_row['item0'] = row['item0']
        participants_row['item1'] = row['item1']
        participants_row['item2'] = row['item2']
        participants_row['item3'] = row['item3']
        participants_row['item4'] = row['item4']
        participants_row['item5'] = row['item5']
        participants_row['item6'] = row['item6']
        participants_row['win'] = row['win']
        participants.append(participants_row)

    for row in timeline['info']['frames']:
        # frameCounter = 1
        # frame = row["{:02d}".format(frameCounter)]
        # frameCounter += 1
        for frameRow in row['events']:
            timeline_row = {}
            timeline_row['matchId'] = matchId
            eventCounter = 0
            # eventFrame = frameRow["{:02d}".format(eventCounter)]
            eventFrame = frameRow
            eventCounter += 1

            if eventFrame['type'] == 'CHAMPION_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['victimId'] = eventFrame['victimId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                if 'bounty' in eventFrame:
                    timeline_row['bounty'] = eventFrame['bounty']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'BUILDING_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['teamId']
                timeline_row['buildingType'] = eventFrame['buildingType']
                timeline_row['laneType'] = eventFrame['laneType']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ITEM_PURCHASED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['itemId'] = eventFrame['itemId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_PLACED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['creatorId'] = eventFrame['creatorId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'LEVEL_UP':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['level'] = eventFrame['level']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ELITE_MONSTER_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['killerTeamId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                timeline_row['monsterType'] = eventFrame['monsterType']
                matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0);
participantsDf
matchesDf
matchTimelineDf


,matchId,type,participantId,level,timestamp,itemId,killerId,victimId,assistingParticipantIds,bounty,positionX,positionY,teamId,buildingType,laneType,wardType,creatorId,monsterType,timestampMIN
0,EUW1_5757717687,LEVEL_UP,1.0,2.0,532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,EUW1_5757717687,LEVEL_UP,1.0,3.0,532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,EUW1_5757717687,LEVEL_UP,2.0,2.0,532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,EUW1_5757717687,LEVEL_UP,2.0,3.0,532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,EUW1_5757717687,LEVEL_UP,3.0,2.0,532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10269,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1546576,NaN,3.0,7.0,"[1, 2, 4, 5]",163.0,6554.0,6417.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0
10270,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1548591,NaN,3.0,9.0,"[1, 2, 4, 5]",180.0,7546.0,6713.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0
10271,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1550177,NaN,3.0,6.0,[2],180.0,7891.0,7203.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0
10272,EUW1_5738045046,ITEM_PURCHASED,9.0,NaN,1553645,2140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0


In [49]:
matchTimelineDf

,matchId,type,participantId,level,timestamp,itemId,killerId,victimId,assistingParticipantIds,bounty,positionX,positionY,teamId,buildingType,laneType,wardType,creatorId,monsterType,timestampMIN
0,EUW1_5757717687,LEVEL_UP,1.0,2.0,532,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0
1,EUW1_5757717687,LEVEL_UP,1.0,3.0,532,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0
2,EUW1_5757717687,LEVEL_UP,2.0,2.0,532,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0
3,EUW1_5757717687,LEVEL_UP,2.0,3.0,532,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0
4,EUW1_5757717687,LEVEL_UP,3.0,2.0,532,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10269,EUW1_5738045046,CHAMPION_KILL,nan,nan,1546576,nan,3.0,7.0,"[1, 2, 4, 5]",163.0,6554.0,6417.0,nan,nan,nan,nan,nan,nan,26.0
10270,EUW1_5738045046,CHAMPION_KILL,nan,nan,1548591,nan,3.0,9.0,"[1, 2, 4, 5]",180.0,7546.0,6713.0,nan,nan,nan,nan,nan,nan,26.0
10271,EUW1_5738045046,CHAMPION_KILL,nan,nan,1550177,nan,3.0,6.0,[2],180.0,7891.0,7203.0,nan,nan,nan,nan,nan,nan,26.0
10272,EUW1_5738045046,ITEM_PURCHASED,9.0,nan,1553645,2140.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,26.0


In [47]:
participantsDf

,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5757717687,1,TheChriz,,Pantheon,10/11/10,14,17909,22690,3048,...,4,7,6692,3111,3004,2003,3134,1018,2052,False
1,EUW1_5757717687,2,Holysben,,Sona,1/11/14,13,4504,19376,5287,...,4,13,3107,2031,3070,3158,6617,1033,2052,False
2,EUW1_5757717687,3,chrisbdx,,Irelia,9/13/6,14,14644,25669,1053,...,32,4,3065,0,0,6673,3153,3111,0,False
3,EUW1_5757717687,4,AramApplication,,Alistar,7/8/17,15,9414,24607,5919,...,32,4,3111,2031,6662,4401,3083,0,2052,False
4,EUW1_5757717687,5,BigDoob27,,Seraphine,5/5/21,15,16021,9984,1690,...,4,32,6655,3020,3040,3165,0,0,2052,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,EUW1_5738045046,6,Helpless Damsel,,Ahri,13/13/18,18,30636,30022,6165,...,4,14,6655,3157,3089,4628,4630,3020,0,False
196,EUW1_5738045046,7,SuperYoLoL,,Irelia,12/13/14,18,27324,51730,6927,...,32,4,6673,3047,3153,6333,3065,3748,0,False
197,EUW1_5738045046,8,glorfındell,,Kayn,10/18/13,18,28554,36235,4907,...,32,4,3142,3134,6676,3042,6691,3111,0,False
198,EUW1_5738045046,9,GeneralLie,,Twitch,6/11/19,18,20149,29540,5540,...,4,7,6672,3006,3085,3153,3036,1038,2052,False


In [82]:
participantsDf.loc[(participantsDf["participant"] == 2) & (participantsDf["matchId"] == "EUW1_5757717687")]["summonerName"].values[0]

'Holysben'

In [14]:
# matchTimelineDf.fillna("nan", inplace=True)
participantsDf.loc[(participantsDf["participant"] == 10) & (participantsDf["matchId"] == "EUW1_5757717687")]["summonerName"].values[0]

'LGC VirguleCyką'

In [17]:
def getSummonerName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = int(row["killerId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = int(row["victimId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def getChampionName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = row["killerId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = row["victimId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo


In [16]:
# matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)

0                    NaN
1                    NaN
2                    NaN
3                    NaN
4                    NaN
              ...       
10269         SuperYoLoL
10270         GeneralLie
10271    Helpless Damsel
10272                NaN
10273                NaN
Length: 10274, dtype: object

In [18]:
matchTimelineDf["participantName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf["participantChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "participant"), axis=1)
matchTimelineDf["killerName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf["killerChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "killer"), axis=1)
matchTimelineDf["victimName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf["victimChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "victim"), axis=1)


In [19]:
matchTimelineDf

,matchId,type,participantId,level,timestamp,itemId,killerId,victimId,assistingParticipantIds,bounty,...,wardType,creatorId,monsterType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion
0,EUW1_5757717687,LEVEL_UP,1.0,2.0,532,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,TheChriz,Pantheon,NaN,NaN,NaN,NaN
1,EUW1_5757717687,LEVEL_UP,1.0,3.0,532,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,TheChriz,Pantheon,NaN,NaN,NaN,NaN
2,EUW1_5757717687,LEVEL_UP,2.0,2.0,532,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,Holysben,Sona,NaN,NaN,NaN,NaN
3,EUW1_5757717687,LEVEL_UP,2.0,3.0,532,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,Holysben,Sona,NaN,NaN,NaN,NaN
4,EUW1_5757717687,LEVEL_UP,3.0,2.0,532,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,chrisbdx,Irelia,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10269,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1546576,NaN,3.0,7.0,"[1, 2, 4, 5]",163.0,...,NaN,NaN,NaN,26.0,NaN,NaN,Super Sett Gökü,Gangplank,SuperYoLoL,Irelia
10270,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1548591,NaN,3.0,9.0,"[1, 2, 4, 5]",180.0,...,NaN,NaN,NaN,26.0,NaN,NaN,Super Sett Gökü,Gangplank,GeneralLie,Twitch
10271,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1550177,NaN,3.0,6.0,[2],180.0,...,NaN,NaN,NaN,26.0,NaN,NaN,Super Sett Gökü,Gangplank,Helpless Damsel,Ahri
10272,EUW1_5738045046,ITEM_PURCHASED,9.0,NaN,1553645,2140.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26.0,GeneralLie,Twitch,NaN,NaN,NaN,NaN


In [20]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
7,EUW1_5757717687,8
17,EUW1_5747889998,8
27,EUW1_5747855273,8
34,EUW1_5745986536,5
49,EUW1_5745972015,10
53,EUW1_5745902193,4
66,EUW1_5745834432,7
77,EUW1_5745362126,8
87,EUW1_5745318060,8
93,EUW1_5745284458,4


In [21]:
kelodyEvents = []
#TODO bool if kelody was assist or not
# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,level,timestamp,itemId,killerId,victimId,assistingParticipantIds,bounty,...,wardType,creatorId,monsterType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion
0,EUW1_5757717687,LEVEL_UP,8.0,2.0,532,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,Kelody,Kassadin,NaN,NaN,NaN,NaN
1,EUW1_5757717687,LEVEL_UP,8.0,3.0,532,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,Kelody,Kassadin,NaN,NaN,NaN,NaN
2,EUW1_5757717687,ITEM_PURCHASED,8.0,NaN,8590,3802.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,Kelody,Kassadin,NaN,NaN,NaN,NaN
3,EUW1_5757717687,CHAMPION_KILL,NaN,NaN,41355,NaN,8.0,3.0,"[9, 10]",270.0,...,NaN,NaN,NaN,1.0,NaN,NaN,Kelody,Kassadin,chrisbdx,Irelia
4,EUW1_5757717687,CHAMPION_KILL,NaN,NaN,73496,NaN,8.0,3.0,"[6, 7, 9, 10]",180.0,...,NaN,NaN,NaN,1.0,NaN,NaN,Kelody,Kassadin,chrisbdx,Irelia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,EUW1_5738045046,ITEM_PURCHASED,10.0,NaN,1330062,2139.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,22.0,Kelody,Diana,NaN,NaN,NaN,NaN
1517,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1415202,NaN,6.0,2.0,[10],180.0,...,NaN,NaN,NaN,24.0,NaN,NaN,Helpless Damsel,Ahri,Ang3lsbl00d,Janna
1518,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1415268,NaN,3.0,10.0,"[1, 2, 5]",180.0,...,NaN,NaN,NaN,24.0,NaN,NaN,Super Sett Gökü,Gangplank,Kelody,Diana
1519,EUW1_5738045046,CHAMPION_KILL,NaN,NaN,1417679,NaN,9.0,1.0,"[6, 10]",180.0,...,NaN,NaN,NaN,24.0,NaN,NaN,GeneralLie,Twitch,Typ82,Jinx


Check how often a Player died in Early Game (pre 10 mins)

In [22]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
7,EUW1_5757717687,8,6
17,EUW1_5747889998,8,0
27,EUW1_5747855273,8,2
34,EUW1_5745986536,5,6
49,EUW1_5745972015,10,5
53,EUW1_5745902193,4,7
66,EUW1_5745834432,7,4
77,EUW1_5745362126,8,3
87,EUW1_5745318060,8,1
93,EUW1_5745284458,4,8


Features:
Died from ganks in laning phase pre 15 min (check ID from enemy Jungler)

Died with ... Gold

Died from 1v1s on lane

Kills from 1v1s on lane

Good roaming (assists pre 15 min)

Warding (check ward placements pre 15 min) maybe in combination with ganks

CS per min (if data available)

Tower Kills/Assists

Objective Kills/Assists (Baron, Dragon, Herald)



Features on the tableau visualization:

Timeline

Champion Icons

In [24]:
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
